# Beans Disease classification: Transfer Learning

The goal is to build a robust machine learning model that is able to distinguish between diseases in the Bean plants. Beans are an important cereal food crop for Africa grown by many small-holder farmers - they are a significant source of proteins for school-age going children in East Africa.

The data is of leaf images representing 3 classes: the healthy class of images, and two disease classes including Angular Leaf Spot and Bean Rust diseases. The model should be able to distinguish between these 3 classes with high accuracy. The end goal is to build a robust, model that can be deployed on a mobile device and used in the field by a farmer.

In [1]:
#TODO: improve description (images?)

## Environment

In [2]:
# TODO: move to source files

from enum import Enum, auto

class DatasetSource(Enum):
    HUGGING_FACE = auto()
    TENSORFLOW = auto()
    KAGGLE = auto()

class BaseModel(Enum):
    XCEPTION = auto()
    EFFICIENT_NET_V2 = auto()
    MOBILE_NET = auto()

class Optimizer(Enum):
    SGD = auto()
    ADAM = auto()
    NADAM = auto()

### Configuration

In [3]:
# TODO: keep all configuration at the same place (temporarily here but eventually in config file)

PREPROCESS_IN_MODEL = False

DATASET_SOURCE = DatasetSource.TENSORFLOW
BASE_MODEL = BaseModel.XCEPTION
OPTIMIZER = Optimizer.SGD

# Same ratio as in original dataset
# TODO: use percentage in case we use datasets with different sizes
TRAIN_SIZE = 1034
VAL_SIZE = 133
TEST_SIZE = 128

BATCH_SIZE = 16
EPOCHS_PRETRAIN = 5 # 20
EPOCHS_FINETUNE = 10 # 100

### Mixed precision

In [4]:
# WARNING: Do not use Mixed Precision for mobile - TFLite doesn't support float16 for all ops yet
# https://ai.google.dev/edge/litert/models/ops_compatibility#supported_types

# policy = tf.keras.mixed_precision.Policy("mixed_float16")
# tf.keras.mixed_precision.set_global_policy(policy)

### GPU Memory

In [5]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

2025-09-24 11:38:19.081780: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


1 Physical GPUs, 1 Logical GPUs


I0000 00:00:1758706710.198802    3939 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2874 MB memory:  -> device: 0, name: Quadro P600, pci bus id: 0000:01:00.0, compute capability: 6.1


### Imports

In [6]:
import tensorflow as tf

#TODO: Download dataset from Kaggle
if (DATASET_SOURCE == DatasetSource.TENSORFLOW
    or DATASET_SOURCE == DatasetSource.KAGGLE):
    import tensorflow_datasets as tfds
elif (DATASET_SOURCE == DatasetSource.HUGGING_FACE):
    from datasets import load_dataset, concatenate_datasets

import keras
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input, Resizing, Lambda, RandomFlip, RandomRotation, RandomContrast
from keras.models import Sequential, Model
from keras.optimizers import SGD, Adam, Nadam
from keras.optimizers.schedules import CosineDecayRestarts
from keras.callbacks import EarlyStopping, TensorBoard

if (BASE_MODEL == BaseModel.XCEPTION):
    from keras.applications.xception import Xception, preprocess_input
elif (BASE_MODEL == BaseModel.EFFICIENT_NET_V2):
    from keras.applications.efficientnet_v2 import EfficientNetV2S, preprocess_input
elif (BASE_MODEL == BaseModel.MOBILE_NET):
    from keras.applications.mobilenet import MobileNet, preprocess_input

from sklearn.model_selection import train_test_split

import numpy as np

import collections

import matplotlib.pyplot as plt

import datetime

### Versions

Check versions for compatibility issues.

In [7]:
# TODO: requirements file should be enough and could replace this

from importlib.metadata import version

def print_version(module_name):
    print(f"{module_name}: {version(module_name)}")

print_version("tensorflow")
print_version("keras")
print_version("scikit-learn")
print_version("numpy")

tensorflow: 2.20.0
keras: 3.11.3
scikit-learn: 1.7.2
numpy: 2.3.3


In [8]:
!nvidia-smi

Wed Sep 24 11:38:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.170                Driver Version: 573.44         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Quadro P600                    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   64C    P0            N/A  / 5001W |      33MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
!nvcc --version

/usr/bin/bash: line 1: nvcc: command not found


### Determinism

Make sure every run is deterministic so that results are reproducible and improvements verified.

In [10]:
# Set the seed using keras.utils.set_random_seed. This will set:
# 1) `numpy` seed
# 2) backend random seed
# 3) `python` random seed
keras.utils.set_random_seed(42)

# If using TensorFlow, this will make GPU ops as deterministic as possible,
# but it will affect the overall performance, so be mindful of that.
# TODO: this is causing heavy performance hit - see how much different are results without it
# tf.config.experimental.enable_op_determinism()

In [11]:
start_time = datetime.datetime.now()

## Loading data

Load everything so that we can split it ourselves.

In [12]:
if (DATASET_SOURCE == DatasetSource.TENSORFLOW
    or DATASET_SOURCE == DatasetSource.KAGGLE):
    ds, info = tfds.load(
        "beans",
        split="all",
        with_info=True,
        shuffle_files=True,
        as_supervised=True,
    )
    ds_all = tfds.as_dataframe(ds, info)
    ds_all_images = ds_all["image"]
    ds_all_labels = ds_all["label"]

    n_classes = info.features["label"].num_classes
    class_names = info.features["label"].names

elif (DATASET_SOURCE == DatasetSource.HUGGING_FACE):
    ds = load_dataset("AI-Lab-Makerere/beans")
    ds_all = concatenate_datasets([ds["train"], ds["validation"], ds["test"]])
    ds_all_images = ds_all["image"]
    ds_all_labels = ds_all["labels"]

    class_names = ["angular_leaf_spot", "bean_rust", "healthy"]
    n_classes = len(class_names)


2025-09-24 11:38:34.539723: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:396] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2025-09-24 11:38:35.148871: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Stratified split

Original split caused huge difference in performance of model when doing predictions with validation and test data sets.

In [ ]:
train_data, temp_data, train_labels, temp_labels = train_test_split(
    ds_all_images,
    ds_all_labels,
    train_size=TRAIN_SIZE,
    random_state=42,
    shuffle=True,
    stratify=ds_all_labels,
)

val_data, test_data, val_labels, test_labels = train_test_split(
    temp_data,
    temp_labels,
    train_size=VAL_SIZE,
    random_state=42,
    shuffle=True,
    stratify=temp_labels,
)

ds_train = tf.data.Dataset.from_tensor_slices((np.array(train_data.tolist()), np.array(train_labels.tolist())))
ds_valid = tf.data.Dataset.from_tensor_slices((np.array(val_data.tolist()), np.array(val_labels.tolist())))
ds_test = tf.data.Dataset.from_tensor_slices((np.array(test_data.tolist()), np.array(test_labels.tolist())))

2025-09-24 11:38:40.017535: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 3102000000 exceeds 10% of free system memory.
2025-09-24 11:38:53.824268: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.89GiB (rounded to 3102000128)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-09-24 11:38:53.824291: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1049] BFCAllocator dump for GPU_0_bfc
2025-09-24 11:38:53.824299: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1056] Bin (256): 	Total Chunks: 11, Chunks in use: 11. 2.8KiB allocated for chunks. 2.8KiB in use in bin. 136B client-requested in use in bin.
2025-09-24 11:38:53.824303: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

Make sure original ratio is preserved.

In [ ]:
print(f"train set {len(list(ds_train))}")
print(f"validation set {len(list(ds_valid))}")
print(f"test set {len(list(ds_test))}")

Make sure that ratio of different classes is same across datasets

In [ ]:
def count_labels(dataset):
    label_counts = collections.Counter()

    for _, label in dataset:
        label_counts[label.numpy()] += 1

    total_count = sum(label_counts.values())
    label_ratios = {label: count / total_count for label, count in label_counts.items()}
    
    sorted_label_ratios = dict(sorted(label_ratios.items()))

    return sorted_label_ratios

print(count_labels(ds_train))
print(count_labels(ds_valid))
print(count_labels(ds_test))

## Preprocess and Augmentation

In [ ]:
preprocess = Sequential([
    Resizing(height=224, width=224, crop_to_aspect_ratio=True),
    Lambda(preprocess_input)
])

# TODO: instead of augmenting existing sample - try creating new ones with augmentation
# (not too many since that can cause overfitting as well)
# TODO: test more augmentations (that make sense) and different parameters (e.g. zoom, brightness, noise)
preprocess_and_augmentation = Sequential([
    Resizing(height=224, width=224, crop_to_aspect_ratio=True),
    RandomFlip(mode="horizontal", seed=42),
    RandomRotation(factor=0.05, seed=42),
    RandomContrast(factor=0.2, seed=42),
    Lambda(preprocess_input)
])
augmentation = Sequential([
    RandomFlip(mode="horizontal", seed=42),
    RandomRotation(factor=0.05, seed=42),
    RandomContrast(factor=0.2, seed=42),
])

resizing_and_augmentation = Sequential([
    Resizing(height=224, width=224, crop_to_aspect_ratio=True),
    RandomFlip(mode="horizontal", seed=42),
    RandomRotation(factor=0.05, seed=42),
    RandomContrast(factor=0.2, seed=42)
])

## Input pipeline

Preprocess and augmentation is part of input pipeline because it is faster.

When processing power / time is not an issue, I recommend moving it to model.
Apart from cleaner and more flexible code, having it as part of model actually improved performance a bit.

In [ ]:
if (not PREPROCESS_IN_MODEL):
    ds_train = ds_train.map(
        lambda X, y: (preprocess(X), y), num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache("cache_train")
ds_train = ds_train.shuffle(TRAIN_SIZE)
if (not PREPROCESS_IN_MODEL):
    ds_train = ds_train.map(
        lambda X, y: (augmentation(X), y), num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [ ]:
if (not PREPROCESS_IN_MODEL):
    ds_valid = ds_valid.map(
        lambda X, y: (preprocess(X), y), num_parallel_calls=tf.data.AUTOTUNE)
ds_valid = ds_valid.cache("cache_val")
ds_valid = ds_valid.batch(BATCH_SIZE)
ds_valid = ds_valid.prefetch(tf.data.AUTOTUNE)

In [ ]:
if (not PREPROCESS_IN_MODEL):
    ds_test = ds_test.map(
        lambda X, y: (preprocess(X), y), num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.cache("cache_test")
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

Verify that augmentation produces photos similar to original dataset (too much augmentation might cause model not to generalize well).

In [ ]:
plt.figure(figsize=(12, 12))
for X_batch, y_batch in ds_train.take(1):
    # TODO: make sure we get clear augmented and original images even if preprocessing in model
    # X_batch = resizing_and_augmentation(X_batch)
    # X_batch = preprocess_input(X_batch)
    for index in range(9):
        plt.subplot(3, 3, index + 1)
        plt.imshow((X_batch[index] + 1) / 2)  # rescale to 0–1 for imshow()
        plt.title(f"Class: {class_names[y_batch[index]]}")
        plt.axis("off")

plt.show()

## Base model with pre-trained weights

TODO: Create a separate README for experiment results / conclusions

Xception with SGD optimizer converges pretty quickly with good results (other models needed more time with lower learning to achieve same performance).

This version mostly follows example from [Machine Learning Notebooks, 3rd edition](https://github.com/ageron/handson-ml3).

In [ ]:
if BASE_MODEL == BaseModel.XCEPTION:
    base_model = Xception(input_shape=(224, 224, 3), include_top=False)
elif BASE_MODEL == BaseModel.EFFICIENT_NET_V2:
    base_model = EfficientNetV2S(input_shape=(224, 224, 3), include_top=False)
elif BASE_MODEL == BaseModel.MOBILE_NET:
    base_model = MobileNet(input_shape=(224, 224, 3), include_top=False)

This simple top performs surprisingly well (however Dropout is needed for overfitting). 

Adding additional Dense layer, Batch Normalization layer and/or Dropout layer required more hyperparameter tuning to get similar performance (but never as good.).

In [ ]:
if (PREPROCESS_IN_MODEL):
    inputs = Input(shape=(None, None, 3))
    x = resizing_and_augmentation(inputs)
    x = preprocess_input(x)
    x = base_model(x, training=False)
else:
    inputs = base_model.input
    x = base_model.output

x = GlobalAveragePooling2D()(x)
# TODO: test different dropout
x = Dropout(0.5)(x)

# TODO: test kernel_regularizer
outputs = Dense(n_classes, activation="softmax")(x)
model = Model(inputs=inputs, outputs=outputs)

Since we are using base_model layers directly (base_model.output), we need to iterate through all of them and switch training off.

In [ ]:
# When preprocessing is done in model, we do not need to set trainable flag per base layer
if PREPROCESS_IN_MODEL:
    base_model.trainable = False
else:
    for layer in base_model.layers:
        layer.trainable = False

## Initial training without touching base model

Early stopping is here in this version more to make sure that the best weights are used.

In different versions where we needed to train for longer, it also helped us to end early when there was not progress (and avoid overfitting).

In [ ]:
early_stopping = EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True, mode="min"
)

In [ ]:
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath="model_checkpoint.keras",
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
)

SGD works surprisingly well - doesn't use as much memory, converges fast and with good results.

Adam required more training and had bigger memory footprint with much smaller learning rate (and/or learning rate scheduler).

In [ ]:
if Optimizer.SGD:
    optimizer = SGD(learning_rate=0.1, momentum=0.9)
elif Optimizer.Adam:
    optimizer = Adam()
elif OPTIMIZER == Optimizer.NADAM:
    optimizer = Nadam(learning_rate=1e-4)

model.compile(
    loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
)

In [ ]:
import gc
import psutil

class RAMMonitor(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()  # Force garbage collection
        ram_percent = psutil.virtual_memory().percent
        print(f"Epoch {epoch}: RAM usage {ram_percent:.1f}%")

In [ ]:
history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS_PRETRAIN,
    callbacks=[early_stopping, model_checkpoint_callback, RAMMonitor()],
)

In [ ]:
restored_epoch = early_stopping.best_epoch + 1 # EarlyStopping uses 0-based indexing (from observation)
restored_epoch

In [ ]:
test_loss, test_acc = model.evaluate(ds_test, verbose=2)
print(test_acc)

In [ ]:
acc_history = history.history["accuracy"]
val_acc_history = history.history["val_accuracy"]

loss_history = history.history["loss"]
val_loss_history = history.history["val_loss"]

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc_history, label="Training Accuracy")
plt.plot(val_acc_history, label="Validation Accuracy")
plt.legend(loc="lower right")
plt.xticks(range(100))
plt.ylabel("Accuracy")
plt.ylim([min(plt.ylim()), max(plt.ylim())])
plt.plot([restored_epoch - 1, restored_epoch - 1], plt.ylim(), label="Best weights")
plt.title("Training and Validation Accuracy")

plt.subplot(2, 1, 2)
plt.plot(loss_history, label="Training Loss")
plt.plot(val_loss_history, label="Validation Loss")
plt.legend(loc="upper right")
plt.xticks(range(100))
plt.ylabel("Cross Entropy")
plt.ylim([min(plt.ylim()), max(plt.ylim())])
plt.plot([restored_epoch - 1, restored_epoch - 1], plt.ylim(), label="Best weights")
plt.title("Training and Validation Loss")
plt.xlabel("epoch")
plt.show()

## Fine tuning

Same as before - early stopping is used mostly to get best weights.

Using bigger number for patience as we fine tune for longer.

In [ ]:
early_stopping = EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True, mode="min"
)

# TensorBoard Callback (repeating here to have unique timestamp)
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# TODO: test different learning rate schedules (e.g. one cycle scheduler, ReduceLROnPlateau)
lr_schedule = CosineDecayRestarts(
    initial_learning_rate=1e-4,  # Adjust if needed
    first_decay_steps=20,        # Increased from 10
    t_mul=2.0,                   # You can try increasing this
    m_mul=1.0,                   # You can experiment with values < 1.0
    alpha=0.0                    # Adjust if needed
)

Unfreeze some layers from base_model for fine tuning. We also need to use much lower learning rate so that existing weights are not completely destroyed.

In [ ]:
if PREPROCESS_IN_MODEL:
    base_model.trainable = True

    for layer in base_model.layers[:-226]:
        layer.trainable = False
else:
    for layer in base_model.layers[56:]:
        layer.trainable = True

if OPTIMIZER == Optimizer.SGD:
    # TODO: test lr schedule for SGD as well (different learning rates though)
    optimizer = SGD(learning_rate=0.01, momentum=0.9)
elif OPTIMIZER == Optimizer.ADAM:
    optimizer = Adam(learning_rate=lr_schedule)
elif OPTIMIZER == Optimizer.NADAM:
    # TODO: test lr schedule for NADAM as well (different learning rates though)
    optimizer = Nadam(learning_rate=1e-4)

model.compile(
    loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
)

In [ ]:
history_fine = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS_FINETUNE,
    callbacks=[early_stopping, tensorboard_callback, model_checkpoint_callback, RAMMonitor()],
)

In [ ]:
restored_epoch_fine = early_stopping.best_epoch + 1 # EarlyStopping uses 0-based indexing (from observation)
restored_epoch_fine

In [ ]:
train_loss, train_acc = model.evaluate(ds_train, verbose=2)
print(train_acc)

In [ ]:
val_loss, val_acc = model.evaluate(ds_valid, verbose=2)
print(val_acc)

In [ ]:
test_loss, test_acc = model.evaluate(ds_test, verbose=2)
print(test_acc)

Dismiss history that was not used (Early Stopping)

In [ ]:
acc_history = acc_history[:restored_epoch]
val_acc_history = val_acc_history[:restored_epoch]

loss_history = loss_history[:restored_epoch]
val_loss_history = val_loss_history[:restored_epoch]

In [ ]:
acc_history += history_fine.history["accuracy"]
val_acc_history += history_fine.history["val_accuracy"]

loss_history += history_fine.history["loss"]
val_loss_history += history_fine.history["val_loss"]

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc_history, label="Training Accuracy")
plt.plot(val_acc_history, label="Validation Accuracy")
plt.xticks(range(100))
plt.ylim([min(plt.ylim()), max(plt.ylim())])
plt.plot(
    [restored_epoch - 1, restored_epoch - 1], plt.ylim(), label="Start Fine Tuning"
)
plt.plot(
    [
        restored_epoch + restored_epoch_fine - 1,
        restored_epoch + restored_epoch_fine - 1,
    ],
    plt.ylim(),
    label="Best weights",
)
plt.legend(loc="lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(2, 1, 2)
plt.plot(loss_history, label="Training Loss")
plt.plot(val_loss_history, label="Validation Loss")
plt.xticks(range(100))
plt.ylim([min(plt.ylim()), max(plt.ylim())])
plt.plot(
    [restored_epoch - 1, restored_epoch - 1], plt.ylim(), label="Start Fine Tuning"
)
plt.plot(
    [
        restored_epoch + restored_epoch_fine - 1,
        restored_epoch + restored_epoch_fine - 1,
    ],
    plt.ylim(),
    label="Best weights",
)
plt.legend(loc="upper right")
plt.title("Training and Validation Loss")
plt.xlabel("epoch")
plt.show()

## Save model

In [ ]:
model.save("beans_disease_classification_transfer_learning.keras")

In [ ]:
duration = datetime.datetime.now() - start_time
print(duration)

## Verify that config prevailed

In [ ]:
tf.keras.mixed_precision.global_policy()

In [ ]:
for gpu in tf.config.list_physical_devices('GPU'):
    print(gpu)
    if tf.config.experimental.get_device_details(gpu):
        print(tf.config.experimental.get_device_details(gpu))

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print(f"Memory growth enabled: {tf.config.experimental.get_memory_growth(gpus[0])}")
    
    # Also check current memory usage
    try:
        memory_info = tf.config.experimental.get_memory_info('GPU:0')
        print(f"Current: {memory_info['current']/1024**3:.2f}GB")
        print(f"Peak: {memory_info['peak']/1024**3:.2f}GB")
    except:
        print("Memory info not available during execution")

## Prepare model for mobile

In [ ]:
model = keras.models.load_model("beans_disease_classification_transfer_learning.keras")

### Test Model on Samples

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np

class_names = {
    0 : "Angular Leaf",
    1 : "Bean Rust",
    2 : "Healthy"
}

# Path to local samples
samples_dir = "hugging_face/samples"

# Get all sample images
sample_files = [f for f in os.listdir(samples_dir) if f.endswith(('.jpg', '.png'))]
sample_files.sort()  # Sort for consistent ordering

print(f"Found {len(sample_files)} sample images: {sample_files}")

# Create a mapping from filename to expected class
filename_to_class = {
    "angular_leaf_spot_01.jpg": 0,  # angular_leaf_spot
    "bean_rust_01.jpg": 1,          # bean_rust  
    "healthy_01.jpg": 2,             # healthy
}

# Test inference on all sample images
fig, axes = plt.subplots(1, len(sample_files), figsize=(15, 5))
if len(sample_files) == 1:
    axes = [axes]

for idx, filename in enumerate(sample_files):
    # Load image
    img_path = os.path.join(samples_dir, filename)
    sample_image = Image.open(img_path).convert("RGB")
    true_label = filename_to_class.get(filename, -1)
    
    # Resize image to model input size
    sample_image_resized = sample_image.resize((224, 224))
    
    # Display the image
    axes[idx].imshow(sample_image_resized)
    axes[idx].set_title(f"{filename}\nTrue: {class_names[true_label] if true_label != -1 else 'Unknown'}")
    axes[idx].axis("off")
    
    # Preprocess for inference
    img_array = np.array(sample_image_resized)
    img_batch = np.expand_dims(img_array, axis=0)
    
    # Apply preprocessing
    preprocessed_img = preprocess(img_batch)
    
    # Perform inference
    predictions = model.predict(preprocessed_img, verbose=0)
    predicted_class = np.argmax(predictions[0])
    confidence = np.max(predictions[0])
    
    print(f"\n--- {filename} ---")
    print(f"True label: {class_names[true_label] if true_label != -1 else 'Unknown'}")
    print(f"Predicted label: {class_names[predicted_class]}")
    print(f"Confidence: {confidence:.4f}")
    print(f"Correct: {'✓' if predicted_class == true_label else '✗'}")
    print(f"Prediction probabilities:")
    for i, prob in enumerate(predictions[0]):
        print(f"  {class_names[i]}: {prob:.4f}")

plt.tight_layout()
plt.show()

### Convert to TFLite

In [ ]:
# Convert Keras model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Optional: Enable optimizations for smaller model size
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Force float32 input/output types for compatibility
converter.target_spec.supported_types = [tf.float32]
converter.inference_input_type = tf.float32
converter.inference_output_type = tf.float32

# Enable Select TF Ops fallback for unsupported ops (required for Xception)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS     # enable TensorFlow ops.
]

tflite_model = converter.convert()

# Save the TFLite model
with open('beans_disease_classification_transfer_learning.tflite', 'wb') as f:
    f.write(tflite_model)

print(f"TensorFlow Lite model saved as 'beans_disease_classification_transfer_learning.tflite'")
print(f"Model size: {len(tflite_model) / 1024 / 1024:.2f} MB")